In [ ]:
"""
BERT 입력으로 뭐가 들어가는지
BERT seq_len 으로 뭐가 들어가야하는지

"""


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import json
import sys
sys.path.append('..')

In [4]:
train_data = pd.read_table('data/ratings_train.txt')
test_data = pd.read_table('data/ratings_test.txt')

In [5]:
"""
데이터 확인
"""
train_data[:5]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [6]:
"""
데이터 정제
"""
print('훈련용 리뷰 개수 :',len(train_data))

훈련용 리뷰 개수 : 150000


In [7]:
train_data['document'].nunique(), train_data['label'].nunique()

(146182, 2)

In [8]:
train_data.drop_duplicates(subset=['document'], inplace=True)

In [9]:
print(train_data.isnull().values.any())

True


In [10]:
print(train_data.isnull().sum())

id          0
document    1
label       0
dtype: int64


In [11]:
train_data.loc[train_data.document.isnull()]

,id,document,label
25857,2172111,NaN,1


In [12]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [13]:
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9 ]","")
train_data[:5]

<ipython-input-13-5581c5b3ca4c>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9 ]","")


,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [14]:
train_data['document'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

id            0
document    313
label         0
dtype: int64


In [15]:
train_data = train_data.dropna(how = 'any')
print(len(train_data))

145869


In [3]:
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9 ]","") # 정규 표현식 수행
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))


NameError: name 'test_data' is not defined

In [30]:
from tokenization_kobert import KoBertTokenizer

In [40]:
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")

In [41]:
a=tokenizer.encode("안녕하세요 여러분 저는 영윤이입니다. 반가워요 여러분",max_length = 128, padding = True)

In [42]:
max_len = 128

In [43]:
len(a)

18

In [44]:
a.extend(([1])*(max_len-len(a)))

In [45]:
a

[2,
 3135,
 5724,
 7814,
 3305,
 3990,
 5760,
 3376,
 7068,
 7096,
 7139,
 54,
 2207,
 5330,
 7018,
 6999,
 3305,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [46]:
len(a)

128

In [23]:
def prepare_with_json(infile, outfile):
    with open(outfile, 'w')as f:
        for index, row in infile.iterrows():
            doc = tokenizer.encode(row["document"])
            instance = {"id": row["id"], "doc":row["document"], "label":row["label"]}
            f.write(json.dumps(instance))
            f.write("\n")

In [24]:
prepare_with_json(train_data, "data/ratings_train.json")
prepare_with_json(test_data, "data/ratings_test.json")

In [20]:
train_data

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1
...,...,...,...
149995,6222902,인간이 문제지 소는 뭔죄인가,0
149996,8549745,평점이 너무 낮아서,1
149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,2376369,청춘 영화의 최고봉방황과 우울했던 날들의 자화상,1
